# Monterey Flipper Pinball Standings Projections

## Purpose
This notebook is used to project the final standings of the Monterey Flipper Pinball league.
The highest 5 weekly scores for each player are summed at the end of the season to produce a winner.
There are 10 weeks total, so you can drop up to 5 scores if you attend every week.

As of right now it is the end of week 4, my initial idea is to take your average score and use that to replace any lower scores, as well as the correct number of future weeks.

## TODO/Issues
- Possibly redo converting the json to a dataframe, there has to be a better way
- Projections fail if the average is taken over a player's best games
- API fails to pull all players in Winter 17 MFP league, (series_id = 475)

In [15]:
import pandas as pd
import numpy as np
import requests
import json
from collections import defaultdict

In [16]:
# MatchPlay API with the Series ID for MFP Spring 18
series_id = '582'
series_url = 'https://matchplay.events/data/series/'+series_id+'/standings'

In [17]:
# Call the API
r = requests.get(series_url)
data = r.json()

In [18]:
# Split up the response, which gives overall standings and also weekly scores (tournament_points)
overall = data['overall']
overall = pd.DataFrame.from_records(overall)
tournament_points = data['tournament_points']

In [19]:
# Janky code to rearrange the tournament_points json response

points = defaultdict(list)
num_weeks = 0
for week in tournament_points:
    if week['status'] != 'planned':
        num_weeks += 1
        for player in week['points'].items():
            points[(player[0],week['tournament_id'])].append(player[1])


In [20]:
# Changing the points dict into a tidy dataframe

points_df = pd.DataFrame(points)
points_df = points_df.transpose().reset_index(1).pivot(columns='level_1',values=0)

In [21]:
# Url to grab weekly standings for names
week_ids = points_df.columns[:num_weeks]
tournament_url_a = 'https://matchplay.events/data/tournaments/'
tournament_url_b = '/standings'

In [22]:
player_names = {}
for week_id in week_ids:
    tournament_url = tournament_url_a + str(week_id) + tournament_url_b
    r = requests.get(tournament_url)
    data = r.json()
    for player in data:
        if player['player_id'] not in player_names:
            player_names[str(player['player_id'])] = player['name']

In [23]:
points_df.index = points_df.index.to_series().map(player_names)

In [24]:
# Apply function that takes the row and does necessary computations for extra columns
weeks_left = 10-num_weeks

def compute_projections(row):
    # Compute average score of all weeks
    row['avg_score'] = row[:num_weeks].mean()
    
    # Number of weeks played
    row['num_games'] = row[:num_weeks].count()
    
    # Total number of points accumulated
    row['cur_total'] = sum([points for points in row[:num_weeks] if points > 0])

    # Find best five games for a player
    best_games = [points for points in row[:num_weeks] if points > 0]
    best_games.sort(reverse=True)
    best_games = best_games[:5]
    
    # Find a player's current score, the metric by which players are judged at the end of the season
    row['cur_score'] = sum(best_games)
    
    # Compute average score of a player's best 5 weeks
    #row['avg_score'] = row['cur_score']/len(best_games)
    
    # Exclude a players
    best_games_aa = [points for points in best_games if points >= row['avg_score']]
    best_games_aa.sort(reverse=True)    
    
    # mean_adj_score: the points added to a player's projection to supplant their below average games
    # initialize variable to 0 for players that have 5 games above their average
    #     this may be deprecated by finding the average of the 5 best games instead of all games
    row['mean_adj_score'] = 0
    
    # If a player has five games above their average, no games are replaced by that average
    if len(best_games_aa) >= 5:
        row['kept_score'] = sum(best_games_aa[:5])
        row['proj_score'] = row['kept_score']
        
    # Replace weeks below average with the average score, depending on how many weeks are left
    if len(best_games_aa) < 5:
        row['kept_score'] = sum(best_games_aa)
        
        # replace the weeks below average to fill to 5, or however many weeks are left if it wouldn't be possible
        if (5-len(best_games_aa) <= weeks_left):
            row['mean_adj_score'] = (5-len(best_games_aa))*row['avg_score']
        else:
            row['kept_score'] = sum(best_games[:-weeks_left])
            row['mean_adj_score'] = weeks_left*row['avg_score']
        
        # Project final score based on player's average
        row['proj_score'] = row['kept_score'] + row['mean_adj_score']
    
    # Find a player's maximum possible score assuming they score a perfect 35 every remaining week
    if weeks_left < 5:
        row['max_score_possible'] = sum(best_games[:5-weeks_left]) + weeks_left*35
    else:
        row['max_score_possible'] = 5*35
    
    # The number of weeks of average score added to a player's projection,
    #   so the number of weeks the player would need to do above average to reach their projection
    row['remaining_weeks_above_avg'] = (row['mean_adj_score']/row['avg_score'])
    
    return row


In [25]:
# Apply the calculations
final_points_df = points_df.apply(compute_projections, axis=1)

In [26]:
current_leader = final_points_df.cur_score.max()

# calculate the average a player must get in the remaining games to beat the leader
def compute_required_avg(row):
    
    
    return row

In [27]:
final_points_df = final_points_df.apply(compute_required_avg, axis=1)

In [29]:
final_points_df.sort_values('proj_score', ascending = False, inplace=True)